# 🏗️ Application Load Balancer con Cloud Armor

En este laboratorio, configurarás un **Application Load Balancer (ALB)** global en Google Cloud, implementado en los **Points of Presence (POP)** de la red global de Google. Además, utilizarás **Cloud Armor** para proteger el tráfico mediante políticas de seguridad basadas en listas de IP permitidas o bloqueadas. Este enfoque es ideal para proteger aplicaciones empresariales, como sistemas contables, asegurando baja latencia, alta disponibilidad y seguridad frente a accesos maliciosos.

💡 **Beneficio empresarial**: El uso de un ALB con **Cloud Armor** permite a las empresas optimizar la distribución del tráfico, garantizar la disponibilidad de servicios críticos (como plataformas de contabilidad) y proteger datos sensibles frente a ataques, reduciendo costos asociados a interrupciones o brechas de seguridad.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Google Cloud Armor](https://cloud.google.com/armor/docs).

## 🎯 Objetivos de aprendizaje
- 🔐 Crear reglas de firewall para HTTP y **health checks**.
- 🖥️ Configurar dos **instance templates**.
- 👥 Crear dos **managed instance groups (MIG)**.
- 🌐 Configurar un **Application Load Balancer** con soporte para IPv4 e IPv6.
- 🧪 Ejecutar una prueba de carga (**stress test**) sobre el ALB.
- 🚫 Bloquear una dirección IP con **Cloud Armor**.

📊 **Concepto clave**: Los **Points of Presence (POP)** son nodos de borde donde Google recibe tráfico externo y lo enruta a través de su red privada global, minimizando latencia y permitiendo la mitigación temprana de ataques.

## 🛠️ Tarea 1: Configurar reglas de firewall para HTTP y health checks

En esta tarea, crearás reglas de firewall para permitir tráfico HTTP y los **health checks** necesarios para que el ALB funcione correctamente.

### Paso 1.1: Regla HTTP (default-allow-http)

1. En **Google Cloud Console**, ve a **VPC network** > **Firewall**.
2. Haz clic en **Create Firewall Rule**.
3. Configura:
   - **Network**: `default`
   - **Targets**: **Specified target tags**
   - **Target tags**: `http-server`
   - **Source IPv4 ranges**: `0.0.0.0/0` (permite todo Internet)
   - **Protocols/ports**: `TCP 80`
4. Haz clic en **Create**.

**Explicación**:
- Esta regla permite tráfico HTTP (puerto 80) hacia las instancias con la etiqueta `http-server`.

💡 **Contexto empresarial**: Esta regla es como autorizar el acceso público a un sistema contable en línea, asegurando que solo los servidores designados puedan recibir solicitudes.

### Paso 1.2: Regla Health Check (default-allow-health-check)

1. En la página de **Firewall**, haz clic en **Create Firewall Rule**.
2. Configura:
   - **Target tags**: `http-server`
   - **Source IPv4 ranges**: `130.211.0.0/22 35.191.0.0/16` (introduce uno, espacio, luego el otro)
   - **Protocols**: `TCP` (todos los puertos)
3. Haz clic en **Create**.

**Explicación**:
- Los **health checks** verifican la disponibilidad de las instancias. Si una instancia no responde, el ALB deja de enviarle tráfico, garantizando alta disponibilidad.
- Los rangos de IP especificados son los utilizados por los **health checks** de Google Cloud.

💡 **Contexto empresarial**: Los **health checks** son como auditorías automáticas que verifican si un sistema financiero está operativo, asegurando que los usuarios siempre puedan acceder a los servicios.

## 🛠️ Tarea 2: Configurar instance templates y crear grupos de instancias

En esta tarea, crearás plantillas de instancias (**instance templates**) y grupos de instancias gestionados (**MIG**) para escalar automáticamente los servidores web.

### Paso 2.1: Crear instance template Region 1-template

1. Ve a **Compute Engine** > **Instance templates** > **Create**.
2. Configura:
   - **Name**: `Region 1-template`
   - **Series**: `E2`
   - **Machine type**: `e2-micro`
   - **Advanced options** > **Networking** > **Network tags**: `http-server`
   - **Network interfaces**: `default` (Region 1)
   - **Management** > **Metadata** > **+ADD ITEM**:
     - **Key**: `startup-script-url`
     - **Value**: `gs://cloud-training/gcpnet/httplb/startup.sh`
3. Haz clic en **Create**.

**Explicación**:
- El **startup script** instala **Apache** y personaliza la página de bienvenida mostrando la IP del cliente, el hostname y la zona.

💡 **Contexto empresarial**: El **startup script** es como un procedimiento automatizado que configura un sistema contable con información específica al iniciarse, facilitando auditorías.

### Paso 2.2: Duplicar template para Region 2

1. Selecciona `Region 1-template` > **+ CREATE SIMILAR**.
2. Cambia el **Name** a `Region 2-template`.
3. En **Network interfaces**, selecciona la subred `default` (Region 2).
4. Mantén el tag `http-server` y haz clic en **Create**.

**Explicación**:
- Esto crea una plantilla similar para una región diferente, asegurando que las instancias estén distribuidas geográficamente.

### Paso 2.3: Crear Managed Instance Groups (MIG)

#### Grupo Region 1-mig

1. Ve a **Compute Engine** > **Instance groups** > **Create instance group**.
2. Configura:
   - **Instance template**: `Region 1-template`
   - **Location**: **Multiple zones** (Region 1)
   - **Min instances**: `1`
   - **Max instances**: `2`
   - **Autoscaling signal**: **CPU utilization 80%**
   - **Initialization period**: `45 s`
3. Haz clic en **Create**.

#### Grupo Region 2-mig

Repite el proceso con:
- **Instance template**: `Region 2-template`
- **Location**: **Multiple zones** (Region 2)

**Explicación**:
- Los **MIG** escalan automáticamente según la carga (CPU, RPS, etc.), asegurando disponibilidad y optimizando costos.

💡 **Contexto empresarial**: Un **MIG** es como un equipo contable que se amplía automáticamente durante picos de trabajo, como cierres fiscales, para garantizar eficiencia.

#### Verificación rápida

1. Ve a **Compute Engine** > **VM instances** y verifica que existan instancias con nombres que comienzan por `Region 1-mig` y `Region 2-mig`.
2. Haz clic en sus **IP externas** y observa la página web generada por el **startup script**.

Para más información, consulta la [Documentación de Google Cloud Compute Engine](https://cloud.google.com/compute/docs).

## 🛠️ Tarea 3: Configurar el Application Load Balancer

En esta tarea, configurarás un **Application Load Balancer (ALB)** global para distribuir tráfico entre los **MIG** creados.

### Paso 3.1: Crear el ALB

1. Ve a **Navigation menu** > **VIEW ALL PRODUCTS** > **Networking** > **Network Services** > **Load balancing**.
2. Haz clic en **Create load balancer**.
3. Selecciona **Application Load Balancer HTTP(S)** > **Next**.
4. Elige **Public facing (externo)** > **Next**.
5. Selecciona **Best for global workloads** > **Next**.
6. Selecciona **Global external Application Load Balancer** > **Next**.
7. Configura:
   - **Load Balancer Name**: `http-lb`
8. Haz clic en **Configure**.

### Paso 3.2: Frontend (IPv4 e IPv6)

Configura dos frontends:

| **Propiedad** | **Valor IPv4** | **Valor IPv6** |
|---------------|----------------|----------------|
| **Protocol**  | HTTP           | HTTP           |
| **IP version**| IPv4           | IPv6           |
| **IP address**| Ephemeral      | Auto-allocate  |
| **Port**      | 80             | 80             |

### Paso 3.3: Backend service (http-backend)

1. Configura el backend:
   - **Instance group**: `Region 1-mig`
     - **Port**: `80`
     - **Balancing mode**: `Rate`
     - **Max RPS**: `50`
     - **Capacity**: `100%`
2. Añade otro backend:
   - **Instance group**: `Region 2-mig`
     - **Balancing mode**: `Utilization`
     - **Max utilization**: `80%`
     - **Capacity**: `100%`
3. Crea un **health check**:
   - **Name**: `http-health-check`
   - **Protocol**: `TCP`
   - **Port**: `80`
   - **Check interval**: `5 s`
   - **Unhealthy threshold**: `2 failures`
4. Activa **Logging** con **Sample Rate** = `1`.
5. Haz clic en **Review and Finalize** > **Create**.
6. Anota las direcciones **IPv4 ([LB_IP_v4])** e **IPv6 ([LB_IP_v6])** del balanceador.

**Explicación**:
- El ALB distribuye tráfico al backend más cercano según latencia y capacidad, con políticas de desbordamiento si un backend se satura.

💡 **Contexto empresarial**: El ALB es como un sistema de distribución automática que asegura que las solicitudes a un sistema contable se procesen en el servidor disponible más cercano, optimizando tiempos de respuesta.

## 🛠️ Tarea 4: Probar el Load Balancer

En esta tarea, probarás el funcionamiento del ALB mediante acceso básico y una prueba de carga (**stress test**).

### Paso 4.1: Acceso básico

1. En tu navegador, accede a:
   - `http://[LB_IP_v4]`
   - `http://[LB_IP_v6]` (si tu red soporta IPv6)
2. Observa la página generada por el **startup script** (puede tardar unos minutos).

**Explicación**:
- Esto verifica que el ALB está distribuyendo tráfico correctamente a los backends.

### Paso 4.2: Prueba de carga (siege)

1. Ve a **Compute Engine** > **VM instances** > **Create instance**.
2. Configura:
   - **Name**: `siege-vm`
   - **Region**: `Region 3` (por ejemplo, `us-west1`)
   - **Zone**: `Zone 3` (por ejemplo, `us-west1-a`)
   - **Series**: `E2`
3. Cuando la VM esté lista, haz clic en **SSH**.
4. Instala **siege**:

In [ ]:
sudo apt-get -y install siege

**Explicación del comando**:
- `sudo`: Ejecuta con privilegios de superusuario.
- `apt-get`: Gestor de paquetes de Debian/Ubuntu.
- `-y`: Acepta automáticamente la instalación.
- `install siege`: Instala la herramienta **siege** para pruebas de carga HTTP.

5. Exporta la IP del Load Balancer:

In [ ]:
export LB_IP=[LB_IP_v4]

**Explicación**:
- `export`: Crea una variable de entorno.
- `LB_IP`: Almacena la IP IPv4 del ALB para usarla en comandos posteriores.

6. Ejecuta la prueba de carga:

In [ ]:
siege -c 150 -t120s http://$LB_IP

**Explicación del comando**:
- `siege`: Herramienta para pruebas de carga HTTP.
- `-c 150`: Simula 150 usuarios concurrentes.
- `-t120s`: Ejecuta la prueba durante 120 segundos.
- `http://$LB_IP`: URL objetivo (usa la variable `LB_IP`).

7. Mientras se ejecuta, observa el tráfico en **Load balancing** > **http-backend** > **Monitoring**.
   - Inicialmente, el tráfico se distribuye a `Region 1-mig`.
   - Si las RPS aumentan, también se usará `Region 2-mig`.
8. Detén **siege** con **CTRL + C** y revisa las estadísticas.

💡 **Contexto empresarial**: La prueba de carga simula un alto volumen de usuarios accediendo a un sistema contable, verificando que el ALB puede manejar picos de demanda sin interrupciones.

## 🛠️ Tarea 5: Bloquear la IP de siege-vm con Cloud Armor

En esta tarea, crearás una política de **Cloud Armor** para bloquear la IP de `siege-vm`, simulando la protección contra un ataque.

### Paso 5.1: Crear política denylist-siege

1. Anota la **External IP** de `siege-vm` (`[SIEGE_IP]`).
2. Ve a **Network Security** > **Cloud Armor policies** > **Create policy**.
3. Configura:
   - **Name**: `denylist-siege`
   - **Default rule action**: `Allow`
4. Haz clic en **Next step**.
5. Añade una regla:
   - **Condition > Match**: `[SIEGE_IP]`
   - **Action**: `Deny`
   - **Response code**: `403 (Forbidden)`
   - **Priority**: `1000`
6. Haz clic en **Save Change to Rule** > **Next step** > **Add Target**.
7. Configura:
   - **Type**: `Backend service (external ALB)`
   - **Target**: `http-backend`
   - Selecciona **Replace** si se solicita.
8. Haz clic en **Create policy**.

**Explicación**:
- La política bloquea el tráfico desde la IP de `siege-vm`, protegiendo el ALB de solicitudes no deseadas.

💡 **Contexto empresarial**: **Cloud Armor** es como un sistema de seguridad que bloquea accesos no autorizados a los registros financieros, evitando ataques que podrían interrumpir operaciones.

### Paso 5.2: Verificar bloqueo

1. En la sesión **SSH** de `siege-vm`, ejecuta:

In [ ]:
curl http://$LB_IP

**Resultado esperado**:
- `403 Forbidden`

2. Desde tu navegador local, accede a `http://[LB_IP_v4]`.
   - ✅ Debería funcionar, ya que la regla por defecto es `Allow`.

3. Vuelve a ejecutar la prueba de carga en `siege-vm`:

In [ ]:
siege -c 150 -t120s http://$LB_IP

**Resultado esperado**:
- No habrá salida, ya que las peticiones son bloqueadas por **Cloud Armor**.

4. Revisa los logs en **Cloud Armor** > **denylist-siege** > **Logs** > **View policy logs**.
   - Filtra por `resource: http-lb-forwarding-rule`.
   - Cada entrada mostrará `configuredAction: DENY` y la política `denylist-siege`.

**Explicación**:
- **Cloud Armor** bloquea el tráfico antes de que llegue a la red VPC, facilitando auditorías y detección de patrones anómalos.

💡 **Contexto empresarial**: Los logs de **Cloud Armor** son como un registro de auditoría que documenta intentos de acceso no autorizados, ayudando a cumplir con normativas de seguridad.

## 📚 Glosario rápido

| **Término**       | **Definición breve**                                              |
|-------------------|-------------------------------------------------------------------|
| **POP**           | Punto de presencia de Google en el borde de su red global.        |
| **ALB**           | Application Load Balancer global de capa 7.                       |
| **Health check**  | Sondeo de salud para decidir si enviar tráfico a una instancia.   |
| **MIG**           | Grupo de instancias gestionado con escalado automático.           |
| **Siege**         | Herramienta CLI para pruebas de carga HTTP.                       |
| **Cloud Armor**   | Servicio de WAF/DDoS que aplica políticas de seguridad basadas en reglas. |

## 🚀 Resumen de conceptos

| **Concepto**            | **Explicación contable simplificada**                              |
|-------------------------|-------------------------------------------------------------------|
| **Application Load Balancer** | Sistema que distribuye solicitudes financieras a servidores disponibles. |
| **Cloud Armor**         | Guardia de seguridad que bloquea accesos no autorizados a datos financieros. |
| **Managed Instance Group** | Equipo contable que se expande automáticamente durante picos de trabajo. |
| **Health check**        | Auditoría automática para verificar la operatividad de sistemas.   |
| **Points of Presence**  | Oficinas regionales que reciben y procesan solicitudes rápidamente. |

💡 **Conclusión empresarial**: Configurar un **Application Load Balancer** con **Cloud Armor** permite a las empresas garantizar la disponibilidad y seguridad de sistemas críticos, como los de contabilidad, mientras optimizan costos mediante el escalado automático y la protección contra ataques. Los logs de **Cloud Armor** facilitan auditorías, asegurando el cumplimiento normativo.

Para más información, consulta la [Documentación de Google Cloud Load Balancing](https://cloud.google.com/load-balancing/docs) y la [Documentación de Google Cloud Armor](https://cloud.google.com/armor/docs).